### 단계 1) 테이블 생성하기

In [ ]:
import sqlite3

# sqlite3를 사용하여 데이터베이스를 생성하고 연결하기
dbpath = 'data/test.sqlite'
conn = sqlite3.connect(dbpath)

# 테이블을 생성하고 데이터 넣기
c = conn.cursor()
c.executescript(
    """
    /* item 테이블이 이미 있으면 삭제 */
    DROP TABLE IF EXISTS daum_news;

    CREATE TABLE items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        link TEXT,
    );
    """
)

# 데이터베이스에 반영하기
conn.commit()

# 데이터베이스 연결 닫기
conn.close()

### 단계2) 수집하기

In [3]:
import requests
from bs4 import BeautifulSoup
import time

page = 3 # 수집하길 원는 페이지 분량
newslist = [] # 데이터를 저장할 리스트

for i in range(1, page):
  url = 'https://news.daum.net/digital#'+str(i)

  res = requests.get(url)
  soup = BeautifulSoup(res.text, 'html.parser')

  articles = soup.select('.list_newsmajor .link_txt')
  for article in articles:
    newslist.append(
      (article.text,article.get('href'))
    )

  time.sleep(1)

### 데이터 SQL에 추가하기

In [ ]:
conn = sqlite3.connect(dbpath)

c = conn.cursor()
c.executemany('INSERT INTO items (title, link) VALUES (?, ?)', newslist)
conn.commit()
conn.close()